In [2]:
# ! .venv\Scripts\pip install darts
# ! .venv\Scripts\pip install dask dask[distributed]
# ! .venv\Scripts\pip install bokeh

In [1]:
import pandas as pd
df = pd.read_parquet('TRAIN_Reco_2021_2022_2023.parquet.gzip').reset_index()

In [3]:
df['ExecutionTime'] = pd.to_datetime(df['ExecutionTime'])
numerical_columns = ['high', 'low', 'close', 'volume']
df[numerical_columns] = df[numerical_columns].astype('float16')

In [4]:
df_2023 = df[df['ExecutionTime'].dt.year == 2023]

In [ ]:
# df_2022.set_index('ExecutionTime', inplace=True)

# def create_lag_rolling_features(df):
#     # Lag features
#     for column in ['low', 'high', 'close', 'volume']:
#         for lag in range(1, 11):  # Create 10 lags
#             df[f'{column}_lag_{lag}'] = df[column].shift(lag)
    
#     # Rolling window features (rolling mean of the last 10 periods)
#     for column in ['low', 'high', 'close', 'volume']:
#         df[f'{column}_rolling_mean_10'] = df[column].rolling(window=10).mean()
    
#     return df

# # Apply the lag and rolling window function to each asset group separately
# df_2022 = df_2022.groupby('ID').apply(create_lag_rolling_features)

In [8]:
train_start_date = '2022-01-01'
train_end_date = '2023-06-30'

val_start_date = '2023-07-01'
val_end_date = '2023-12-31'  # Adjust if you have data beyond 2023

# Step 4: Split the data into training and validation sets
train_df = df[(df['ExecutionTime'] >= train_start_date) & (df['ExecutionTime'] <= train_end_date)]
val_df = df[(df['ExecutionTime'] >= val_start_date) & (df['ExecutionTime'] <= val_end_date)]

In [10]:
val_df["ID"].nunique()

672

In [6]:
df_2023.set_index('ExecutionTime', inplace=True)

def create_lag_rolling_features(df):
    # Lag features
    for column in ['low', 'high', 'close', 'volume']:
        for lag in range(1, 11):  # Create 10 lags
            df[f'{column}_lag_{lag}'] = df[column].shift(lag)
    
    # Rolling window features (rolling mean of the last 10 periods)
    for column in ['low', 'high', 'close', 'volume']:
        df[f'{column}_rolling_mean_10'] = df[column].rolling(window=10).mean()
    
    return df

# Apply the lag and rolling window function to each asset group separately
df_2023 = df_2023.groupby('ID').apply(create_lag_rolling_features)

C:\Users\a4293604\AppData\Local\Temp\ipykernel_2532\4074872974.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_2023 = df_2023.groupby('ID').apply(create_lag_rolling_features)


In [7]:
df_2023 = df_2023.dropna()
df_2023.head()

ID  high  low  close  volume  \
ID      ExecutionTime                                                  
Fri00Q1 2023-01-01 02:30:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2023-01-01 02:45:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2023-01-01 03:00:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2023-01-01 03:15:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2023-01-01 03:30:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   

                                   low_lag_1  low_lag_2  low_lag_3  low_lag_4  \
ID      ExecutionTime                                                           
Fri00Q1 2023-01-01 02:30:00+01:00        0.0        0.0        0.0        0.0   
        2023-01-01 02:45:00+01:00        0.0        0.0        0.0        0.0   
        2023-01-01 03:00:00+01:00        0.0        0.0        0.0        0.0   
        2023-01-01 03:15:00+01:00        0.0        0.0        0.0        0.0   
        2023-01-01 03:30:00+01:00        0.0        0.0        0.0        0.0   

                                   low_lag_5  ...  volume_lag_5  volume_lag_6  \
ID      ExecutionTime                         ...                               
Fri00Q1 2023-01-01 02:30:00+01:00        0.0  ...           0.0           0.0   
        2023-01-01 02:45:00+01:00        0.0  ...           0.0           0.0   
        2023-01-01 03:00:00+01:00        0.0  ...           0.0           0.0   
        2023-01-01 03:15:00+01:00        0.0  ...           0.0           0.0   
        2023-01-01 03:30:00+01:00        0.0  ...           0.0           0.0   

                                   volume_lag_7  volume_lag_8  volume_lag_9  \
ID      ExecutionTime                                                         
Fri00Q1 2023-01-01 02:30:00+01:00           0.0           0.0           0.0   
        2023-01-01 02:45:00+01:00           0.0           0.0           0.0   
        2023-01-01 03:00:00+01:00           0.0           0.0           0.0   
        2023-01-01 03:15:00+01:00           0.0           0.0           0.0   
        2023-01-01 03:30:00+01:00           0.0           0.0           0.0   

                                   volume_lag_10  low_rolling_mean_10  \
ID      ExecutionTime                                                   
Fri00Q1 2023-01-01 02:30:00+01:00            0.0                  0.0   
        2023-01-01 02:45:00+01:00            0.0                  0.0   
        2023-01-01 03:00:00+01:00            0.0                  0.0   
        2023-01-01 03:15:00+01:00            0.0                  0.0   
        2023-01-01 03:30:00+01:00            0.0                  0.0   

                                   high_rolling_mean_10  \
ID      ExecutionTime                                     
Fri00Q1 2023-01-01 02:30:00+01:00                   0.0   
        2023-01-01 02:45:00+01:00                   0.0   
        2023-01-01 03:00:00+01:00                   0.0   
        2023-01-01 03:15:00+01:00                   0.0   
        2023-01-01 03:30:00+01:00                   0.0   

                                   close_rolling_mean_10  \
ID      ExecutionTime                                      
Fri00Q1 2023-01-01 02:30:00+01:00                    0.0   
        2023-01-01 02:45:00+01:00                    0.0   
        2023-01-01 03:00:00+01:00                    0.0   
        2023-01-01 03:15:00+01:00                    0.0   
        2023-01-01 03:30:00+01:00                    0.0   

                                   volume_rolling_mean_10  
ID      ExecutionTime                                      
Fri00Q1 2023-01-01 02:30:00+01:00                     0.0  
        2023-01-01 02:45:00+01:00                     0.0  
        2023-01-01 03:00:00+01:00                     0.0  
        2023-01-01 03:15:00+01:00                     0.0  
        2023-01-01 03:30:00+01:00                     0.0  

[5 rows x 49 columns]

In [8]:
# df_2022 = df_2022.dropna()
# df_2022.head()

ID  high  low  close  volume  \
ID      ExecutionTime                                                  
Fri00Q1 2022-01-01 02:30:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2022-01-01 02:45:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2022-01-01 03:00:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2022-01-01 03:15:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   
        2022-01-01 03:30:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0   

                                   low_lag_1  low_lag_2  low_lag_3  low_lag_4  \
ID      ExecutionTime                                                           
Fri00Q1 2022-01-01 02:30:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 02:45:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:00:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:15:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:30:00+01:00        0.0        0.0        0.0        0.0   

                                   low_lag_5  ...  volume_lag_5  volume_lag_6  \
ID      ExecutionTime                         ...                               
Fri00Q1 2022-01-01 02:30:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 02:45:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:00:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:15:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:30:00+01:00        0.0  ...           0.0           0.0   

                                   volume_lag_7  volume_lag_8  volume_lag_9  \
ID      ExecutionTime                                                         
Fri00Q1 2022-01-01 02:30:00+01:00           0.0           0.0           0.0   
        2022-01-01 02:45:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:00:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:15:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:30:00+01:00           0.0           0.0           0.0   

                                   volume_lag_10  low_rolling_mean_10  \
ID      ExecutionTime                                                   
Fri00Q1 2022-01-01 02:30:00+01:00            0.0                  0.0   
        2022-01-01 02:45:00+01:00            0.0                  0.0   
        2022-01-01 03:00:00+01:00            0.0                  0.0   
        2022-01-01 03:15:00+01:00            0.0                  0.0   
        2022-01-01 03:30:00+01:00            0.0                  0.0   

                                   high_rolling_mean_10  \
ID      ExecutionTime                                     
Fri00Q1 2022-01-01 02:30:00+01:00                   0.0   
        2022-01-01 02:45:00+01:00                   0.0   
        2022-01-01 03:00:00+01:00                   0.0   
        2022-01-01 03:15:00+01:00                   0.0   
        2022-01-01 03:30:00+01:00                   0.0   

                                   close_rolling_mean_10  \
ID      ExecutionTime                                      
Fri00Q1 2022-01-01 02:30:00+01:00                    0.0   
        2022-01-01 02:45:00+01:00                    0.0   
        2022-01-01 03:00:00+01:00                    0.0   
        2022-01-01 03:15:00+01:00                    0.0   
        2022-01-01 03:30:00+01:00                    0.0   

                                   volume_rolling_mean_10  
ID      ExecutionTime                                      
Fri00Q1 2022-01-01 02:30:00+01:00                     0.0  
        2022-01-01 02:45:00+01:00                     0.0  
        2022-01-01 03:00:00+01:00                     0.0  
        2022-01-01 03:15:00+01:00                     0.0  
        2022-01-01 03:30:00+01:00                     0.0  

[5 rows x 49 columns]

In [50]:
from sklearn.preprocessing import MinMaxScaler

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply the scaler to each asset separately for 2022 (Training)
df_scaled_2022 = df_2022.copy()  # Create a copy of the original dataframe

# List of columns to scale
columns_to_scale = ['high', 'low', 'close', 'volume']

# Apply scaling for each asset (group by 'ID')
for asset in df_scaled_2022.index.get_level_values('ID').unique():
    asset_data_2022 = df_scaled_2022.loc[asset, columns_to_scale]  # Select data for each asset
    scaled_values_2022 = scaler.fit_transform(asset_data_2022)  # Fit and transform the data for this asset (2022)
    df_scaled_2022.loc[asset, columns_to_scale] = scaled_values_2022  # Replace original data with scaled values

In [51]:
# Now apply the same scaler to 2023 (Validation)
df_scaled_2023 = df_2023.copy()  # Create a copy of the 2023 dataframe

for asset in df_scaled_2023.index.get_level_values('ID').unique():
    asset_data_2023 = df_scaled_2023.loc[asset, columns_to_scale]  # Select data for each asset
    scaled_values_2023 = scaler.transform(asset_data_2023)  # Only transform, don't fit again
    df_scaled_2023.loc[asset, columns_to_scale] = scaled_values_2023  # Replace original data with scaled values

In [52]:
df_scaled_2022.head()

ID  high      low  close  volume  \
ID      ExecutionTime                                                      
Fri00Q1 2022-01-01 02:30:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 02:45:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 03:00:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 03:15:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 03:30:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   

                                   low_lag_1  low_lag_2  low_lag_3  low_lag_4  \
ID      ExecutionTime                                                           
Fri00Q1 2022-01-01 02:30:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 02:45:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:00:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:15:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:30:00+01:00        0.0        0.0        0.0        0.0   

                                   low_lag_5  ...  volume_lag_5  volume_lag_6  \
ID      ExecutionTime                         ...                               
Fri00Q1 2022-01-01 02:30:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 02:45:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:00:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:15:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:30:00+01:00        0.0  ...           0.0           0.0   

                                   volume_lag_7  volume_lag_8  volume_lag_9  \
ID      ExecutionTime                                                         
Fri00Q1 2022-01-01 02:30:00+01:00           0.0           0.0           0.0   
        2022-01-01 02:45:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:00:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:15:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:30:00+01:00           0.0           0.0           0.0   

                                   volume_lag_10  low_rolling_mean_10  \
ID      ExecutionTime                                                   
Fri00Q1 2022-01-01 02:30:00+01:00            0.0                  0.0   
        2022-01-01 02:45:00+01:00            0.0                  0.0   
        2022-01-01 03:00:00+01:00            0.0                  0.0   
        2022-01-01 03:15:00+01:00            0.0                  0.0   
        2022-01-01 03:30:00+01:00            0.0                  0.0   

                                   high_rolling_mean_10  \
ID      ExecutionTime                                     
Fri00Q1 2022-01-01 02:30:00+01:00                   0.0   
        2022-01-01 02:45:00+01:00                   0.0   
        2022-01-01 03:00:00+01:00                   0.0   
        2022-01-01 03:15:00+01:00                   0.0   
        2022-01-01 03:30:00+01:00                   0.0   

                                   close_rolling_mean_10  \
ID      ExecutionTime                                      
Fri00Q1 2022-01-01 02:30:00+01:00                    0.0   
        2022-01-01 02:45:00+01:00                    0.0   
        2022-01-01 03:00:00+01:00                    0.0   
        2022-01-01 03:15:00+01:00                    0.0   
        2022-01-01 03:30:00+01:00                    0.0   

                                   volume_rolling_mean_10  
ID      ExecutionTime                                      
Fri00Q1 2022-01-01 02:30:00+01:00                     0.0  
        2022-01-01 02:45:00+01:00                     0.0  
        2022-01-01 03:00:00+01:00                     0.0  
        2022-01-01 03:15:00+01:00                     0.0  
        2022-01-01 03:30:00+01:00                     0.0  

[5 rows x 49 columns]

In [17]:
from darts import TimeSeries
from darts.models import RNNModel

c:\Users\a4293604\Documents\deep-learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
# df_scaled_2022.drop(columns=["ID"], inplace=True)
# df_scaled_2023.drop(columns=["ID"], inplace=True)

In [23]:
# df_scaled_2022 = df_scaled_2022.reset_index()
# df_scaled_2023 = df_scaled_2023.reset_index()

In [53]:

# Convert the DataFrames to Darts TimeSeries objects
def df_to_timeseries(df, value_cols, time_col='ExecutionTime', id_col='ID'):
    time_series_dict = {}
    for asset in df.index.get_level_values(id_col).unique():
        asset_data = df.loc[asset, value_cols].copy()
        asset_time_series = TimeSeries.from_dataframe(asset_data, time_col)
        time_series_dict[asset] = asset_time_series
    return time_series_dict

# Prepare the scaled 2022 data (train) and 2023 data (validation)
value_cols = ['high', 'low', 'close', 'volume']

In [47]:
df_scaled_2022.set_index(['ExecutionTime'], inplace=True)

In [54]:
df_scaled_2022.head()

ID  high      low  close  volume  \
ID      ExecutionTime                                                      
Fri00Q1 2022-01-01 02:30:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 02:45:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 03:00:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 03:15:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   
        2022-01-01 03:30:00+01:00  Fri00Q1   0.0  0.04895    0.0     0.0   

                                   low_lag_1  low_lag_2  low_lag_3  low_lag_4  \
ID      ExecutionTime                                                           
Fri00Q1 2022-01-01 02:30:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 02:45:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:00:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:15:00+01:00        0.0        0.0        0.0        0.0   
        2022-01-01 03:30:00+01:00        0.0        0.0        0.0        0.0   

                                   low_lag_5  ...  volume_lag_5  volume_lag_6  \
ID      ExecutionTime                         ...                               
Fri00Q1 2022-01-01 02:30:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 02:45:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:00:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:15:00+01:00        0.0  ...           0.0           0.0   
        2022-01-01 03:30:00+01:00        0.0  ...           0.0           0.0   

                                   volume_lag_7  volume_lag_8  volume_lag_9  \
ID      ExecutionTime                                                         
Fri00Q1 2022-01-01 02:30:00+01:00           0.0           0.0           0.0   
        2022-01-01 02:45:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:00:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:15:00+01:00           0.0           0.0           0.0   
        2022-01-01 03:30:00+01:00           0.0           0.0           0.0   

                                   volume_lag_10  low_rolling_mean_10  \
ID      ExecutionTime                                                   
Fri00Q1 2022-01-01 02:30:00+01:00            0.0                  0.0   
        2022-01-01 02:45:00+01:00            0.0                  0.0   
        2022-01-01 03:00:00+01:00            0.0                  0.0   
        2022-01-01 03:15:00+01:00            0.0                  0.0   
        2022-01-01 03:30:00+01:00            0.0                  0.0   

                                   high_rolling_mean_10  \
ID      ExecutionTime                                     
Fri00Q1 2022-01-01 02:30:00+01:00                   0.0   
        2022-01-01 02:45:00+01:00                   0.0   
        2022-01-01 03:00:00+01:00                   0.0   
        2022-01-01 03:15:00+01:00                   0.0   
        2022-01-01 03:30:00+01:00                   0.0   

                                   close_rolling_mean_10  \
ID      ExecutionTime                                      
Fri00Q1 2022-01-01 02:30:00+01:00                    0.0   
        2022-01-01 02:45:00+01:00                    0.0   
        2022-01-01 03:00:00+01:00                    0.0   
        2022-01-01 03:15:00+01:00                    0.0   
        2022-01-01 03:30:00+01:00                    0.0   

                                   volume_rolling_mean_10  
ID      ExecutionTime                                      
Fri00Q1 2022-01-01 02:30:00+01:00                     0.0  
        2022-01-01 02:45:00+01:00                     0.0  
        2022-01-01 03:00:00+01:00                     0.0  
        2022-01-01 03:15:00+01:00                     0.0  
        2022-01-01 03:30:00+01:00                     0.0  

[5 rows x 49 columns]

In [55]:
# Convert to TimeSeries (for both train and validation sets)
train_series_dict = df_to_timeseries(df_scaled_2022, value_cols)
# valid_series_dict = df_to_timeseries(df_scaled_2023, value_cols)

# Convert the dictionaries to list of TimeSeries (optional)
train_series_list = list(train_series_dict.values())
# valid_series_list = list(valid_series_dict.values())


AttributeError: time_col='ExecutionTime' is not present.


AttributeError: time_col='ExecutionTime' is not present.

In [ ]:
# Initialize the RNN model
model = RNNModel(
    model="LSTM",                     # You can change this to "RNN" or "GRU" if needed
    input_chunk_length=15,            # The number of past observations to use as input
    output_chunk_length=10,           # The number of future time steps to predict (10 in your case)
    n_epochs=50,                      # Number of epochs (adjust based on your data size)
    random_state=42,                  # Set random state for reproducibility
    batch_size=32                     # Adjust based on memory availability
)

# Train the RNN model on the training data (list of TimeSeries)
model.fit(train_series_list, verbose=True)

# Validate the model using backtesting on the 2023 data
# The model will predict each 10-step forecast and compare it with the actual values in validation data
pred_series_list = []
for valid_series in valid_series_list:
    pred_series = model.historical_forecasts(
        valid_series, 
        start=0.8,                     # Start backtesting after 80% of the data (adjust as needed)
        forecast_horizon=10,           # Forecast horizon of 10 steps (as specified in the task)
        stride=1,                      # Predict every step
        retrain=False,                 # Do not retrain at every step (set to True if retraining is needed)
        verbose=True
    )
    pred_series_list.append(pred_series)

# Evaluate the performance on validation data (sMAPE)
from darts.metrics import smape

for i, valid_series in enumerate(valid_series_list):
    print(f"Validation sMAPE for Asset {i+1}: {smape(valid_series, pred_series_list[i])}")
